In [1]:
import numpy as np
import pandas as pd

In [2]:
articles=pd.read_csv('articles.csv',
                    usecols=['prod_name','product_type_name','product_group_name','graphical_appearance_name',
                              'colour_group_name','perceived_colour_value_name'])
articles.head()

,prod_name,product_type_name,product_group_name,graphical_appearance_name,colour_group_name,perceived_colour_value_name
0,Strap top,Vest top,Garment Upper body,Solid,Black,Dark
1,Strap top,Vest top,Garment Upper body,Solid,White,Light
2,Strap top (1),Vest top,Garment Upper body,Stripe,Off White,Dusty Light
3,OP T-shirt (Idro),Bra,Underwear,Solid,Black,Dark
4,OP T-shirt (Idro),Bra,Underwear,Solid,White,Light


In [3]:
cat_subcat={}
for cat in articles['product_group_name'].unique():
    cat_subcat[cat]=list(articles[articles['product_group_name']==cat]['product_type_name'].unique())

In [4]:
##defining threshold for selecting categories
cat_thres=np.mean(articles['product_group_name'].value_counts().values)

In [5]:
##selecting categories
cats=[cat for cat in articles['product_group_name'].value_counts().index 
      if (articles['product_group_name'].value_counts()[cat]>cat_thres)]

In [6]:
##defining threshold for selecting subcategories
subcat_thres=np.mean(articles['product_type_name'].value_counts().values)

In [7]:
##selecting subcategories
subcats=[subcat for subcat in articles['product_type_name'].value_counts().index 
      if (articles['product_type_name'].value_counts()[subcat]>subcat_thres)]

In [8]:
def no_category(cat):
    if cat not in cats:
        return 'no_category'
    else:
        return cat

In [9]:
def no_subcategory(subcat):
    if subcat not in subcats:
        return 'no_subcategory'
    else:
        return subcat

In [10]:
articles['product_group_name']=articles['product_group_name'].apply(lambda x:no_category(x))
articles['product_type_name']=articles['product_type_name'].apply(lambda x:no_subcategory(x))

In [11]:
articles['product_group_name'].value_counts()

Garment Upper body    42741
Garment Lower body    19812
no_category           18539
Garment Full body     13292
Accessories           11158
Name: product_group_name, dtype: int64

In [12]:
articles['product_type_name'].value_counts()

no_subcategory       11962
Trousers             11169
Dress                10362
Sweater               9302
T-shirt               7904
Top                   4155
Blouse                3979
Jacket                3940
Shorts                3939
Shirt                 3405
Vest top              2991
Underwear bottom      2748
Skirt                 2696
Hoodie                2356
Bra                   2212
Socks                 1889
Leggings/Tights       1878
Sneakers              1621
Cardigan              1550
Hat/beanie            1349
Garment Set           1320
Swimwear bottom       1307
Bag                   1280
Earring               1159
Jumpsuit/Playsuit     1147
Pyjama set            1120
Blazer                1110
Other accessories     1034
Boots                 1028
Scarf                 1013
Bodysuit               913
Hair/alice band        854
Bikini top             850
Name: product_type_name, dtype: int64

In [13]:
def make_df(ordered_cols):
    df=articles
    df['input_text']=df['prod_name']
    ##adding attributes to the product name to generate queries
    ##reversing ordered_cols
    ordered_cols.reverse()
    for col in ordered_cols:
        df['input_text']=df[col]+' '+df['input_text']
    ##reduce to the input and output columns
    df=df[['input_text','product_group_name','product_type_name']]
    ##reducing input text to lowercase
    df['input_text']=df['input_text'].apply(lambda x:x.lower())
    ##return the created dataframe
    return df

In [14]:
##creating combinations
set_1=make_df(['graphical_appearance_name'])
set_2=make_df(['colour_group_name'])
set_3=make_df(['perceived_colour_value_name'])
set_4=make_df(['graphical_appearance_name','colour_group_name'])
set_5=make_df(['perceived_colour_value_name','colour_group_name'])
set_6=make_df(['graphical_appearance_name','perceived_colour_value_name','colour_group_name'])

##deleting the articles dataset
##del(articles)

##concatenating all dataframes into a single one
data=pd.concat([set_1,set_2,set_3,set_4,set_5,set_6])

def jumble_words(sent):
    sent_list=sent.split(' ')
    np.random.shuffle(sent_list)
    return ' '.join(sent_list)

data['input_text']=data['input_text'].apply(lambda x:jumble_words(x))

#resetting index
data.reset_index(inplace=True)
data.drop('index',axis=1,inplace=True)

##deleting all the sets
del(set_1,set_2,set_3,set_4,set_5,set_6)

data.head()

C:\Users\abhsingh73\AppData\Local\Temp\ipykernel_6688\2168342493.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['input_text']=df['input_text'].apply(lambda x:x.lower())
C:\Users\abhsingh73\AppData\Local\Temp\ipykernel_6688\2168342493.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['input_text']=df['input_text'].apply(lambda x:x.lower())
C:\Users\abhsingh73\AppData\Local\Temp\ipykernel_6688\2168342493.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat

,input_text,product_group_name,product_type_name
0,top solid strap,Garment Upper body,Vest top
1,solid strap top,Garment Upper body,Vest top
2,stripe top strap (1),Garment Upper body,Vest top
3,solid t-shirt op (idro),no_category,Bra
4,t-shirt op solid (idro),no_category,Bra


In [15]:
from sklearn.model_selection import StratifiedShuffleSplit

In [16]:
cat_data=data[['input_text','product_group_name']].copy(deep=False)
cat_data['input_text']=cat_data['input_text']+' [opt] '+' [opt] '.join(cats)

cat_sss=StratifiedShuffleSplit(train_size=0.02,random_state=42)

cat_gen=cat_sss.split(cat_data['input_text'],cat_data['product_group_name'])
cat_data=cat_data.iloc[next(cat_gen)[0]]

cat_data.reset_index(inplace=True)
cat_data.drop('index',axis=1,inplace=True)

cat_data.columns=['input_text','target_text']

cat_data['prefix']='translate input_text to target_text'

cat_train_data=cat_data[:10000]
cat_eval_data=cat_data[10000:]

In [18]:
subcat_data=data[['input_text','product_type_name']].copy(deep=False)
subcat_data['input_text']=subcat_data['input_text']+' [opt] '+' [opt] '.join(subcats)

subcat_sss=StratifiedShuffleSplit(train_size=0.02,random_state=42)

subcat_gen=subcat_sss.split(subcat_data['input_text'],subcat_data['product_type_name'])
subcat_data=subcat_data.iloc[next(subcat_gen)[0]]

subcat_data.reset_index(inplace=True)
subcat_data.drop('index',axis=1,inplace=True)

subcat_data.columns=['input_text','target_text']

subcat_data['prefix']='translate input_text to target_text'

subcat_train_data=subcat_data[:10000]
subcat_eval_data=subcat_data[10000:]

In [19]:
import logging
from simpletransformers.t5 import T5Model,T5Args

In [20]:
logging.basicConfig(level=logging.INFO)
transformers_logger=logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [21]:
cat_seq_len=max(cat_data['input_text'].apply(lambda x:len(x.split())))
cat_seq_len

28

In [22]:
subcat_seq_len=max(subcat_data['input_text'].apply(lambda x:len(x.split())))
subcat_seq_len

84

In [23]:
##specifying model arguments
model_args = T5Args()
model_args.max_seq_length=subcat_seq_len
model_args.train_batch_size=128
model_args.eval_batch_size=128
model_args.num_train_epochs=1
##model_args.evaluate_during_training = True
##model_args.evaluate_during_training_steps = 30000
model_args.use_multiprocessing=False
model_args.fp16=False
model_args.save_steps=-1
model_args.save_eval_checkpoints=False
model_args.no_cache=True
model_args.reprocess_input_data=True
model_args.overwrite_output_dir=True
model_args.preprocess_inputs=False
model_args.num_return_sequences=1

##instantiating the model
model=T5Model("t5","t5-small",args=model_args,use_cuda=False)

C:\Users\abhsingh73\Anaconda3\lib\site-packages\transformers\models\t5\tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [24]:
model.train_model(train_data=cat_train_data,eval_data=cat_eval_data)

INFO:simpletransformers.t5.t5_utils: Creating features from dataset file at cache_dir/


  0%|          | 0/10000 [00:00<?, ?it/s]

C:\Users\abhsingh73\Anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:3538: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)
INFO:simpletransformers.t5.t5_model: Training started


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/79 [00:00<?, ?it/s]

INFO:simpletransformers.t5.t5_model: Training of t5-small model complete. Saved to outputs/.


(79, 0.653274255085595)

In [25]:
model.train_model(train_data=subcat_train_data,eval_data=subcat_eval_data)

INFO:simpletransformers.t5.t5_utils: Creating features from dataset file at cache_dir/


  0%|          | 0/10000 [00:00<?, ?it/s]

INFO:simpletransformers.t5.t5_model: Training started


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/79 [00:00<?, ?it/s]

INFO:simpletransformers.t5.t5_model: Training of t5-small model complete. Saved to outputs/.


(79, 0.9463934664484821)

In [27]:
cat_eval_input=list(cat_eval_data['input_text'])
cat_eval_target=list(cat_eval_data['target_text'])
cat_eval_pred=model.predict(cat_eval_input)

Generating outputs:   0%|          | 0/21 [00:00<?, ?it/s]

C:\Users\abhsingh73\Anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:3538: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


Decoding outputs:   0%|          | 0/2665 [00:02<?, ?it/s]

NameError: name 'eval_target' is not defined

In [28]:
cat_acc=sum(1 for target,prediction in zip(cat_eval_target,cat_eval_pred) if target==prediction)/len(cat_eval_input)
cat_acc

0.08930581613508443

In [29]:
subcat_eval_input=list(subcat_eval_data['input_text'])
subcat_eval_target=list(subcat_eval_data['target_text'])
subcat_eval_pred=model.predict(subcat_eval_input)

Generating outputs:   0%|          | 0/21 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/2665 [00:00<?, ?it/s]

In [30]:
subcat_acc=sum(1 for target,prediction in zip(subcat_eval_target,subcat_eval_pred) if target==prediction)/len(subcat_eval_input)
subcat_acc

0.475797373358349

In [32]:
model.predict('shirt'+' [opt] '+' [opt] '.join(cats))

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/1 [00:00<?, ?it/s]

['Garment Lower body']

In [33]:
' [opt] '.join(cats)

'Garment Upper body [opt] Garment Lower body [opt] Garment Full body [opt] Accessories'

In [34]:
model.predict('shirt'+' [opt] '+'Garment Upper body [opt] Garment Full body [opt] Accessories')

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

C:\Users\abhsingh73\Anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:3538: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


Decoding outputs:   0%|          | 0/1 [00:00<?, ?it/s]

['Body Body']

In [36]:
model.predict('skirt'+' [opt] '+'Garment Upper body [opt] Garment Lower body [opt] Accessories')

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/1 [00:00<?, ?it/s]

['Body Body']

In [40]:
model.predict('skirt'+' [opt] '+'Garment Upper body [opt] Garment Full body [opt] Accessories')

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/1 [00:00<?, ?it/s]

['Ski Ski Ski']

In [41]:
model.predict('skirt'+' [opt] '+' [opt] '.join(subcats))

Generating outputs:   0%|          | 0/4 [00:00<?, ?it/s]

C:\Users\abhsingh73\Anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:3538: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


Decoding outputs:   0%|          | 0/4 [00:00<?, ?it/s]

['Skirt', 'Leggings/Tights', 'no no icesuit', 'no_subcategory']

In [42]:
model.predict('shirt'+' [opt] '+' [opt] '.join(subcats))

Generating outputs:   0%|          | 0/4 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/4 [00:00<?, ?it/s]

['Blouse', 'no_subcategory', 'Dress', 'Hair/alice band']

In [44]:
model.save_model(output_dir='t5_scrap')

In [48]:
model=T5Model('t5','outputs/t5_scrap',use_cuda=False)

OSError: outputs/t5_scrap is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo with `use_auth_token` or log in with `huggingface-cli login` and pass `use_auth_token=True`.